# Structures des bandes électroniques
Ce code a pour but d'analyser les **propriétés électroniques** du cristal **MgI2** (mp-23205) sur base des données du site Materials Project [1].

Les images présentées dans ce code sont issues du fichier CIF et du site Materials Project.

La reflexion théorique de ce code est inspirée du cours LMAPR1492-Physique des matériaux donné par le professeur G.M. Riganese [2].

In [1]:
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.ext.matproj import MPRester
with MPRester("ktopgq8J6C3OwUSa") as m:
    
    #Get a BandStructure corresponding to a material_id.
    #Returns : A BandStructure object.
    bstructure = m.get_bandstructure_by_material_id("mp-23205")
    
#Class to plot or get data to facilitate the plot of band structure objects.
bs = BSPlotter(bstructure)

#Get the data nicely formatted for a plot
#Returns : A dictionnary 
bsp = bs.bs_plot_data()


# 0. Zone de Brillouin
"Pour tout vecteur d'onde **k** il est possible de ramener l'énergie propre associée à une énergie d'un point de la première zone de Brillouin"

Sur le shéma ci-dessous, on peut voir la fome de la suface de Fermi dans la première zone de Brillouin (en noir).

Dans le suite de ce code, nous nous attarderons plus en détails sur les points critiques (en bleu) illustrés ci-dessous.

In [2]:
import numpy as np
import matplotlib.pyplot as plt    
#plot the Brillouin zone
bs.plot_brillouin()

<Figure size 640x480 with 1 Axes>

# 1. Bande interdite

La bande interdite est la bande de niveaux d'énergie qui ne peuvent pas être occupés par les électrons. 

Cette dernière est caluclée dans le code suivant : 

In [3]:
# band_gap: A string indicating the band gap and its nature (empty if it’s a metal).
bg = bstructure.get_band_gap()
print(bg)

{'direct': False, 'energy': 3.6162000000000005, 'transition': '\\Gamma-(0.500,0.000,0.444)'}


Pour le MgI2, La bande interdite 
* est **indirecte** : le maximum de la dernière bande de valence n'est pas aligné avec le minimum de la première bande de conduction. 
* vaut **3,6162 eV** : il s'agit d'un isolant électrique
* k évolue selon la direction $\Gamma$ - A|L de la zone de brillouin du cristal.

# 2. Dispertion
Pour calculer le maximum (et le minimum) de dispersion, j'observe la valeur absolue de la pente la plus (et la moins) élevée pour chaque direction.

La pente se calcule comme un rapport de l'énergie sur la distance.

Les direction se traduisent dans le code par:
0. $\Gamma$ - M 
1. M - K 
2. K - $\Gamma$ 
3. $\Gamma$ - A
4. A - L
5. L - H
6. A|L - M|K
7. M|K - H
8. H - $\Gamma$


## 2.1. Dans la dernière bande de valence

In [37]:
import numpy as np
# vbm: A list of tuples (distance,energy) marking the vbms. The energies are shifted with respect to the fermi level is the option has been selected.
# dernière bande de valence : 15

# energy: A dict storing bands for spin up and spin down data [{Spin:[band_index][k_point_index]}] as a list (one element for each branch) of energy for each kpoint. 
#The data is stored by branch to facilitate the plotting. 
e = len(bsp["energy"][0]['1'][15])
print(e)
d = len(bsp["distances"])
print(d)

energie0_g = bsp["energy"][0]['1'][15]
print (energie0_g)
k0_g = bsp["distances"][0]
print (k0_g)
energie0_d = bsp["energy"][e-1]['1'][15]
k0_d = bsp["distances"][d-1]
pente0 = np.abs((energie0_d-energie0_g)/(k0_d-k0_g))
print('pente pour la direction 0:' + str(pente0))

19
9
[7.004900000000001, 7.088600000000001, 7.2379, 7.390500000000001, 7.475200000000001, 7.3702000000000005, 7.2073, 7.196200000000001, 7.269, 7.317500000000001, 7.341700000000001, 7.346300000000001, 7.3544, 7.4443, 7.4984, 7.513400000000001, 7.508000000000001, 7.4992, 7.4955]
[0.0, 0.0478758468708271, 0.0957516937416542, 0.1436275233771778, 0.1915033702480049, 0.23937921711883198, 0.2872550639896591, 0.3351308936251827, 0.38300674049600986, 0.43088258736683693, 0.47875843423766395, 0.5266342811084911, 0.5745101107440147, 0.6223859576148418, 0.6702618044856689, 0.718137651356496, 0.7660134809920196, 0.8138893278628467, 0.8617651747336738]


IndexError: list index out of range

In [34]:
# vbm: A list of tuples (distance,energy) marking the vbms. The energies are shifted with respect to the fermi level is the option has been selected.
# dernière bande de valence : 15

# energy: A dict storing bands for spin up and spin down data [{Spin:[band_index][k_point_index]}] as a list (one element for each branch) of energy for each kpoint. 
#The data is stored by branch to facilitate the plotting. 

k_g = 0 #Valeur du vecteur d'onde k à gauche
energie_g = 0 #Valeur de l'énergie à gauche
k_d = 0 #Valeur du vecteur d'onde k à droite
energie_d = 0 #Valeur de l'énergie à droite
pente_max = 0 #Valeur de la pente maximale
pente_min = 100 #valeur de la pente minimale
k_max  = [0,0] #Paire de valeurs de vecteur d'onde là où se situe la pente maximale
energie_max = [0,0] #Paire de valeur d'énergie où se situe la pente maximale
k_min = [0,0] #Paire de valeurs de vecteur d'onde là où se situe la pente minimale
energie_min = [0,0] #Paire de valeur d'énergie où se situe la pente minimale

#On commence à parcourir les valeurs d'abscisse et d'ordonée possibles. On se concentre sur une branche à la fois.

for k, energie in zip(bsp["distances"], bsp["energy"]):
    k_g = k[0] #La valeur de gauche correspond à la valeur la plus à gauche de la branche
    energie_g = energie['1'][15][0] #La valeur de gauche correspond à la valeur la plus à gauche de la branche. On récupère
                               #l'énergie de chaque branche pour la 15ème bande.
    k_d = k[len(k)-1] #La valeur de droite correspond à la valeur la plus à droite de la branche
    energie_d = energie['1'][15][len(energie['1'][15])-1]    
    pente = np.abs((energie_d-energie_g)/(k_d-k_g)) #Calcul de la valeur absolue de la pente
    if pente >= pente_max:
        pente_max  = pente
        k_max = [k_g,k_d]
        energie_max = [energie_g,energie_d]
    if pente <= pente_min:
        pente_min = pente
        k_min = [k_g,k_d]
        bands_min = [energie_g,energie_d]

print('La pente maximale de la dernière bande de valence se situe entre les abscisses ' + str(k_max) + 
      '\n et entre les ordonnées ' + str(energie_max) + ' et vaut en valeur absolue ' + str(pente_max))
print('La pente minimale de la dernière bande de valence se situe entre les abscisses ' + str(k_min) + 
      '\n et entre les ordonnées ' + str(energie_min) + ' et vaut en valeur absolue ' + str(pente_min))




La pente maximale de la dernière bande de valence se situe entre les abscisses [3.622499933438404, 4.120040298075162]
 et entre les ordonnées [7.5601, 8.3591] et vaut en valeur absolue 1.6058998561520312
La pente minimale de la dernière bande de valence se situe entre les abscisses [5.115120985344404, 5.521469517409459]
 et entre les ordonnées [0, 0] et vaut en valeur absolue 0.1589768263015609


## 2.2. Dans la première bande de conduction

In [38]:
# cbm: A list of tuples (distance,energy) marking the cbms. The energies are shifted with respect to the fermi level is the option has been selected.
# première bande de conduction : 16

# energy: A dict storing bands for spin up and spin down data [{Spin:[band_index][k_point_index]}] as a list (one element for each branch) of energy for each kpoint. 
#The data is stored by branch to facilitate the plotting. 

k_g = 0 #Valeur du vecteur d'onde k à gauche
energie_g = 0 #Valeur de l'énergie à gauche
k_d = 0 #Valeur du vecteur d'onde k à droite
energie_d = 0 #Valeur de l'énergie à droite
pente_max = 0 #Valeur de la pente maximale
pente_min = 100 #valeur de la pente minimale
k_max  = [0,0] #Paire de valeurs de vecteur d'onde là où se situe la pente maximale
energie_max = [0,0] #Paire de valeur d'énergie où se situe la pente maximale
k_min = [0,0] #Paire de valeurs de vecteur d'onde là où se situe la pente minimale
energie_min = [0,0] #Paire de valeur d'énergie où se situe la pente minimale

#On commence à parcourir les valeurs d'abscisse et d'ordonée possibles. On se concentre sur une branche à la fois.

for k, energie in zip(bsp["distances"], bsp["energy"]):
    k_g = k[0] #La valeur de gauche correspond à la valeur la plus à gauche de la branche
    energie_g = energie['1'][16][0] #La valeur de gauche correspond à la valeur la plus à gauche de la branche. On récupère
                               #l'énergie de chaque branche pour la 15ème bande.
    k_d = k[len(k)-1] #La valeur de droite correspond à la valeur la plus à droite de la branche
    energie_d = energie['1'][16][len(energie['1'][16])-1]    
    pente = np.abs((energie_d-energie_g)/(k_d-k_g)) #Calcul de la valeur absolue de la pente
    if pente >= pente_max:
        pente_max  = pente
        k_max = [k_g,k_d]
        energie_max = [energie_g,energie_d]
    if pente <= pente_min:
        pente_min = pente
        k_min = [k_g,k_d]
        bands_min = [energie_g,energie_d]

print('La pente maximale de la première bande de conduction se situe entre les abscisses ' + str(k_max) + 
      '\n et entre les ordonnées ' + str(energie_max) + ' et vaut en valeur absolue ' + str(pente_max))
print('La pente minimale de la première bande de conduction se situe entre les abscisses ' + str(k_min) + 
      '\n et entre les ordonnées ' + str(energie_min) + ' et vaut en valeur absolue ' + str(pente_min))



La pente maximale de la première bande de conduction se situe entre les abscisses [3.622499933438404, 4.120040298075162]
 et entre les ordonnées [9.2344, 8.3625] et vaut en valeur absolue 1.7524206315130877
La pente minimale de la première bande de conduction se situe entre les abscisses [1.3593055393704325, 2.3543862266396727]
 et entre les ordonnées [0, 0] et vaut en valeur absolue 0.06009562919375505


# 3. Masse effective
En supposant une dispertion parabolique.
la masse effective est définie comme : 
$$m^* = \frac{\hbar^2}{\frac{\delta^2 E}{\delta k^2}}$$

## 3.1. Au sommet de la dernière bande de valence

In [44]:
h_bar = 6.582119570 * 1e-16 #eVs
# vbm: A list of tuples (distance,energy) marking the vbms. 
#The energies are shifted with respect to the fermi level is the option has been selected. 
vmb = bstructure.get_vbm()
k = vmb.get('kpoint').frac_coords
k_n = np.linalg.norm(k)
print('La valeur de la norme est ' + str(k_n))
#Valeur de la norme de a
a = 7.763862276302451

#Calcul de la norme de G
G = 2*np.pi/a

#Récupération de l'énergie associée au VBM
E = vmb['energy']

#Calcul de la masse effective
m_e = h_bar**2*G**2/(2*E)
print('La masse effective du sommet de la dernière bande de valence est ' + str(m_e) + ' kg')

La valeur de la norme est [0. 0. 0.]
La masse effective du sommet de la dernière bande de valence est 1.2357366098417937e-31 kg


## 3.2. À la base de la première bande de conduction

In [ ]:
# cbm: A list of tuples (distance,energy) marking the cbms. 
#The energies are shifted with respect to the fermi level is the option has been selected. 
cmb = bstructure.get_cbm()
k = cmb.get('kpoint').frac_coords
k_n = np.linalg.norm(k)

print('La valeur de la norme est ' + str(k_n))

#Récupération de l'énergie associée au CBM
E = cmb['energy']

#Calcul de la masse effective
m_e = h_bar**2*k_n**2/(2*E)

print('La masse effective de la base de la première bande de conduction est ' + str(m_e) + ' kg')


[1] (2020) Materials project mp-23205, [Online] https://materialsproject.org/materials/mp-23205/
[2] (2020) Chapitre 4 : Les électrons dans le cristal, [Online] https://moodleucl.uclouvain.be/pluginfile.php/1274329/mod_resource/content/3/Chap-4.pdf

In [ ]:
# ticks: A dict with the ‘distances’ at which there is a kpoint (the x axis) and the labels (None if no label). 
    # energy: A dict storing bands for spin up and spin down data [{Spin:[band_index][k_point_index]}] as a list (one element for each branch) of energy for each kpoint. The data is stored by branch to facilitate the plotting. 
    # vbm: A list of tuples (distance,energy) marking the vbms. The energies are shifted with respect to the fermi level is the option has been selected. 
    # cbm: A list of tuples (distance,energy) marking the cbms. The energies are shifted with respect to the fermi level is the option has been selected. 
    # lattice: The reciprocal lattice. 
    # zero_energy: This is the energy used as zero for the plot. 
    # is_metal: True if the band structure is metallic (i.e., there is at least one band crossing the fermi level).
    
    
    print("trouver la direction dans laquelle la dernière (première) bande de valence (conduction) présente le plus et le moins de dispersion (insérer une figure indiquant la dispersion par des flèches")
    print("calculer la masse effective au sommet (base) de la dernière (première) bande de valence (conduction) en supposant une dispersion parabolique.")
    
    #Get all ticks and labels for a band structure plot.
    #Returns: A dictionary with 
    # 'distance’: a list of distance at which ticks should be set 
    # 'label’: a list of label for each of those ticks
    bsTricks = bsData.get_ticks()
    
    #Get a matplotlib object for the bandstructure plot. Blue lines are up spin, red lines are down spin.
    bsPlot = bsData.get_plot()